<a href="https://colab.research.google.com/github/kruemmel-python/141024Schule_Einkaufen/blob/master/CipherCore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pycuda


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.7 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=660545 sha256=907997540bc3f1f520078f53c9db24376c24d51e3976cad0138353accc2ccd0d
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [9]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy as np
import time
import re


# Funktion zur Emoji-Substitution
def emoji_substitution(password):
    substitutions = {
      'A': '🔺', 'B': '🔻', 'C': '⚫', 'D': '⚪', 'E': '♦',
      'F': '♠', 'G': '♥', 'H': '♣', 'I': '▲', 'J': '▼',
      'K': '⬛', 'L': '⬜', 'M': '🔶', 'N': '🔷', 'O': '💠',
      'P': '🌟', 'Q': '🌀', 'R': '🔲', 'S': '🔳', 'T': '⚡',
      'U': '☀', 'V': '☁', 'W': '☂', 'X': '☃', 'Y': '☄',
      'Z': '⭐',
      'a': '🍎', 'b': '🍌', 'c': '🍇', 'd': '🍓', 'e': '🍒',
      'f': '🍑', 'g': '🍍', 'h': '🍉', 'i': '🍋', 'j': '🍅',
      'k': '🍆', 'l': '🥭', 'm': '🥝', 'n': '🥥', 'o': '🥑',
      'p': '🌽', 'q': '🌶', 'r': '🍕', 's': '🍔', 't': '🍟',
      'u': '🍫', 'v': '🍩', 'w': '🍰', 'x': '🍪', 'y': '🍿',
      'z': '🍬',
      ' ': ' ',
      '.': '⚫', ',': '⚪',
      '0': '🍏', '1': '🍐', '2': '🍊', '3': '🍋', '4': '🍌',
      '5': '🍉', '6': '🍇', '7': '🍓', '8': '🍒', '9': '🍑'
  }

    substituted_password = ''.join([substitutions.get(char, char) for char in password])
    return substituted_password

# Brute-Force CUDA Kernel
kernel_code = """
__global__ void brute_force(char *charset, char *password, char *result, int charset_len, int pass_len, unsigned long long offset) {
    unsigned long long idx = blockIdx.x * blockDim.x + threadIdx.x + offset;

    // Generiere Passwort-Kombination basierend auf dem Index
    char candidate[100];  // Annahme: max. Passwortlänge von 100
    unsigned long long temp_idx = idx;

    // Generiere die aktuelle Kandidatenkombination
    for (int i = pass_len - 1; i >= 0; i--) {
        candidate[i] = charset[temp_idx % charset_len];
        temp_idx /= charset_len;
    }

    // Vergleiche Kandidaten-Passwort mit dem echten Passwort
    bool match = true;
    for (int i = 0; i < pass_len; i++) {
        if (candidate[i] != password[i]) {
            match = false;
            break;
        }
    }

    // Wenn das Passwort übereinstimmt, speichern wir es im Ergebnis-Array
    if (match) {
        for (int i = 0; i < pass_len; i++) {
            result[i] = password[i];
        }
    }
}
"""

# Brute-Force-Funktion unter Verwendung von PyCUDA
def brute_force_cuda(password, charset):
    # Substitute Emojis im Passwort
    substituted_password = emoji_substitution(password)

    # Konvertiere Charset und Passwort in int32, um größere Zeichen zu unterstützen
    charset_bytes = np.array(list(charset.encode('utf-8')), dtype=np.int32)  # Verwende int32 anstelle von int8
    password_bytes = np.array(list(substituted_password.encode('utf-8')), dtype=np.int32)  # Verwende int32 anstelle von int8

    result = np.zeros_like(password_bytes)  # Ergebnis-Array für das gefundene Passwort

    # PyCUDA Setup
    charset_len = np.int32(len(charset_bytes))
    pass_len = np.int32(len(password_bytes))

    # Statt die gesamte Anzahl der Kombinationen zu berechnen, iteriere über den Suchraum
    max_combinations_per_step = np.uint64(2**32)  # Verarbeite maximal 2^32 Kombinationen pro Schritt

    # Kompiliere den CUDA Kernel
    mod = SourceModule(kernel_code)
    brute_force = mod.get_function("brute_force")

    # Konfiguriere CUDA Block- und Grid-Größen
    threads_per_block = 256
    blocks_per_launch = 1000  # Anzahl der Blöcke pro Ausführung, dies hält es in einem verarbeitbaren Bereich

    print(f"Testing in chunks of {blocks_per_launch * threads_per_block} threads per launch")

    start_time = time.time()
    offset = 0
    while True:
        # Starte den CUDA Kernel mit Offset
        brute_force(
            cuda.In(charset_bytes),
            cuda.In(password_bytes),
            cuda.Out(result),
            charset_len,
            pass_len,
            np.uint64(offset),  # Offset für den aktuellen Schritt
            block=(threads_per_block, 1, 1),
            grid=(blocks_per_launch, 1)
        )

        # Überprüfen, ob wir das Passwort gefunden haben
        found_password = ''.join([chr(c) for c in result if c != 0])
        if found_password == substituted_password:
            end_time = time.time()
            print(f"Passwort gefunden: {found_password} in {end_time - start_time:.4f} Sekunden")
            return

        offset += blocks_per_launch * threads_per_block
        if offset >= max_combinations_per_step:
            print("Alle möglichen Kombinationen wurden getestet.")
            break

    end_time = time.time()
    print(f"Passwort nicht gefunden. Dauer: {end_time - start_time:.4f} Sekunden")

# Beispielaufruf
charset = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
password = "☂🍬🍉@💠🍩🍿🍩🥝@🍕🍉🍍@🍩🍕🥝🍎🍕🍟🍬🍋🍍🍕🍟"  # Beispiel mit Emojis
brute_force_cuda(password, charset)



# Wörterbuchangriff
def dictionary_attack(password, dictionary):
    print(f"Starting Dictionary attack on password: {password}")
    for word in dictionary:
        if word == password:
            print(f"Dictionary attack found password: {word}")
            return word
    print(f"Dictionary attack did not find password: {password}")
    return None

# Mustererkennung
def pattern_recognition(password):
    print(f"Starting Pattern recognition on password: {password}")
    patterns = [
        r"[0-9]{5}",  # Numerische Sequenz mit 5 Ziffern
        r"[A-Za-z]{5}",  # Alphabetische Sequenz mit 5 Buchstaben
        r"[A-Za-z]{2}[0-9]{4}",  # Emoji-Sequenz gefolgt von 4 Ziffern
    ]

    for pattern in patterns:
        if re.match(pattern, password):
            print(f"Pattern recognition found pattern: {pattern}")
            return pattern
    print(f"Pattern recognition did not find any pattern for password: {password}")
    return None

# Kombinierter Angriff
def combined_attack(password, charset, max_length, dictionary):
    # Emoji-Substitution
    substituted_password = emoji_substitution(password)
    print(f"Substituted password: {substituted_password}")

    # Brute-Force-Angriff
    result = brute_force_cuda(substituted_password, charset)
    if result:
        return result

    # Wörterbuchangriff
    result = dictionary_attack(substituted_password, dictionary)
    if result:
        return result

    # Mustererkennung
    result = pattern_recognition(substituted_password)
    if result:
        return result

    return None
# Originales Passwort: 12345
# erste Verschlüsselung (alphabetische Substitution): 87654
# Zweite Verschlüsselung (Emoji/Symbol-Substitution): 🍒🍓🍇🍉🍌
#
# Originales Passwort: Baum
# erste Verschlüsselung (alphabetische Substitution): Yzfn
# Zweite Verschlüsselung (Emoji/Symbol-Substitution): ☄🍬🍑🥥
#
# Originales Passwort: Wind of Change
# erste Verschlüsselung (alphabetische Substitution): Drmw lu Xszmtv
# Zweite Verschlüsselung (Emoji/Symbol-Substitution): ⚪🍕🥝🍰 🥭🍫 ☃🍔🍬🥝🍟🍩
#
# Originales Passwort: Bernd@1988
# erste Verschlüsselung (alphabetische Substitution): Yvimw@8011
# Zweite Verschlüsselung (Emoji/Symbol-Substitution): ⚪🍕🥝🍰 🥭🍫 ☃🍔🍬🥝🍟🍩
# Passwörter und Parameter
passwords = ["🍒🍓🍇🍉🍌", "☄🍬🍑🥥", "⚪🍕🥝🍰 🥭🍫 ☃🍔🍬🥝🍟🍩", "☄🍩🍋🥝🍰@🍒🍏🍐🍐"]

charset = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"
max_length = 10
dictionary = ["12345", "password", "87654", "qwerty"]

# Durchführen des kombinierten Angriffs
for password in passwords:
    print(f"\nAnalyzing password: {password}")
    result = combined_attack(password, charset, max_length, dictionary)
    if result:
        print(f"Passwort gefunden: {result}")
    else:
        print("Passwort nicht gefunden")


Testing in chunks of 256000 threads per launch
Alle möglichen Kombinationen wurden getestet.
Passwort nicht gefunden. Dauer: 7.2354 Sekunden

Analyzing password: 🍒🍓🍇🍉🍌
Substituted password: 🍒🍓🍇🍉🍌
Testing in chunks of 256000 threads per launch
Alle möglichen Kombinationen wurden getestet.
Passwort nicht gefunden. Dauer: 2.1416 Sekunden
Starting Dictionary attack on password: 🍒🍓🍇🍉🍌
Dictionary attack did not find password: 🍒🍓🍇🍉🍌
Starting Pattern recognition on password: 🍒🍓🍇🍉🍌
Pattern recognition did not find any pattern for password: 🍒🍓🍇🍉🍌
Passwort nicht gefunden

Analyzing password: ☄🍬🍑🥥
Substituted password: ☄🍬🍑🥥
Testing in chunks of 256000 threads per launch
Alle möglichen Kombinationen wurden getestet.
Passwort nicht gefunden. Dauer: 2.0979 Sekunden
Starting Dictionary attack on password: ☄🍬🍑🥥
Dictionary attack did not find password: ☄🍬🍑🥥
Starting Pattern recognition on password: ☄🍬🍑🥥
Pattern recognition did not find any pattern for password: ☄🍬🍑🥥
Passwort nicht gefunden

Analyzing pa